In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# Import necessary libraries

# Prepare the data
# Assuming your DataFrame is called 'df' and the target column is 'price'
features = df.drop(['date'], axis=1)  # Remove date column
target = df['price']

# Scale the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Create sequences for LSTM (using last 24 time steps to predict next value)
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length, 0])  # Assuming price is first column after scaling
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_features)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions to original scale
# You'll need to reshape and add other columns back to match original scaling
predictions_reshaped = np.zeros((len(predictions), features.shape[1]))
predictions_reshaped[:, 0] = predictions.flatten()
predictions_original = scaler.inverse_transform(predictions_reshaped)[:, 0]